In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os
# os.chdir('/content/drive/My Drive/Pytorch_Spine_Segmentation')
#
# !pip install albumentations==0.4.6
# !pip install pydicom
# !pip install gdcm
# !pip install pylibjpeg
# !pip install --upgrade numpy
# !pip install pylibjpeg-libjpeg

In [2]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import pylibjpeg
import torch.nn as nn
import torch.optim as optim
from model import UNet
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)

In [3]:
# Hyperparameters and directories
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 64
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "data/train/img/"
TRAIN_MASK_DIR = "data/train/label/"
VAL_IMG_DIR = "data/val/img/"
VAL_MASK_DIR = "data/val/label/"

In [4]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    #여기 어딘가 에러 있는거같은 확인할 것.
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward -- float16
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update(0)

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [5]:
def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0],
                std=[1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0],
                std=[1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNet(in_channels=1, out_channels=1).to(DEVICE) # change out_channel for multi classes
    loss_fn = nn.BCEWithLogitsLoss()  # cross entropy loss for multiple classes
    optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    # if LOAD_MODEL:
    #     load_checkpoint(torch.load("checkpoint.pth.tar"), model)
    # check_accuracy(val_loader, model, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        #save
        checkpoint = {
            "state_dict":model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)
        check_accuracy(val_loader, model, device=DEVICE)

        save_predictions_as_imgs(
            val_loader, model, folder="saved_imgs/", device=DEVICE
        )
    

In [6]:
torch.cuda.is_available()

False

In [ ]:
if __name__ == "__main__":
    main()

  0%|          | 0/2 [00:02<?, ?it/s]


RuntimeError: ignored